In [ ]:
!ls ./drive/MyDrive/2024\ Güz\ Dönemi/veri\ mühendisliği/

'İlerleme Raporu Sosyal Medya.gdoc'   yorumlar.gsheet  'yorumlar - Sayfa1.csv'
'İlerleme Raporu Sosyal Medya.pdf'    yorumlar.ipynb


In [ ]:
from google.colab import files
#uploaded = files.upload()


import pandas as pd

root_dir = "./drive/MyDrive/2024 Güz Dönemi/veri mühendisliği/"

df = pd.read_csv(root_dir + "yorumlar - Sayfa1.csv")
df.columns = ['status', 'comment']

# Veriyi Hazırlama

Veri setimizi model eğitimine uygun hale getirmek için aşağıdaki adımları gerçekleştirmeliyiz:

### 1. Metin Temizleme
İlk olarak, clean_text fonksiyonu ile her bir metin yorumunu temizliyoruz:
* Sentiment analizi yaptığımız için duygunun ifadesine bir katkısı olmadığını düşündüğümüz sayı ve noktalama işaretlerini metinden kaldırıyoruz.
* Tüm yorumları küçük harfe dönüştürüp satır başı ve sonundaki boşlukları kaldırıyoruz.


In [ ]:
import re
def clean_text(text):
    text = re.sub(r'\d+\.', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = text.strip()

    return text


df['comment'] = df['comment'].apply(clean_text)

### 2. Yorumları Vektörleştirme
TF-IDF ve Word2Vec teknikleri kullanılarak her bir yorum bir sayı vektörü haline dönüştürülür.

stop_words: Burada, Türkçe durdurma kelimeleri (örneğin, "ve", "ama", "bir" vb.) kullanılmaktadır. Bu kelimeler anlam taşımadığı için modelde dikkate alınmaz. Bu yüzden metinden ayıklanması gerekmektedir.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

# comment sütunundaki metinleri listeye çevir
comments = df['comment'].astype(str).tolist()

tr_stop_words = ['ve', 'ama', 'bir', 'olarak', 'ile', 'de', 'en', 'çok', 'kadar', 'da', 'için', 'gibi', 'şu', 'bu', 'her', 'herhangi', 'aslında', 'o', 'neden', 'şey', 'ne', 'ya', 'ya da', 'bunu', 'böyle', 'aslında', 'ancak', 'sadece']

# 1. TF-IDF ile vektörleştirme
tfidf_vectorizer = TfidfVectorizer(max_features=300, stop_words=tr_stop_words)  # max_features ile boyut sınırlanabilir
tfidf_matrix = tfidf_vectorizer.fit_transform(comments)

# TF-IDF vektörlerini DataFrame'e ekleyelim
df['TFIDF_Vector'] = list(tfidf_matrix.toarray())

# 2. Word2Vec ile vektörleştirme
# Word2Vec için metinleri kelime listelerine ayırmamız gerekiyor
tokenized_comments = [comment.split() for comment in comments]

# Word2Vec modelini eğit
w2v_model = Word2Vec(sentences=tokenized_comments, vector_size=300, window=5, min_count=1, workers=4)

# Her yorumu Word2Vec ile vektöre dönüştürmek
def get_avg_w2v_vector(sentence):
    vectors = [w2v_model.wv[word] for word in sentence if word in w2v_model.wv]
    if vectors:  # Eğer kelimeler varsa
        return np.mean(vectors, axis=0)
    else:  # Eğer kelimeler modelde yoksa
        return np.zeros(w2v_model.vector_size)

# Her yorumun Word2Vec vektörünü hesapla
w2v_vectors = [get_avg_w2v_vector(comment) for comment in tokenized_comments]
df['Word2Vec_Vector'] = w2v_vectors


In [ ]:
df_copy = df.copy()

# Logistic Regression Modeli

Şimdi hazırlanmış verilerimizi Logistic Regression modeli eğitmek için kullanacağız. TF-IDF vektörlerinin ve Word2Vec vektörlerinin kullanıldığı iki ayrı model eğitip, hangi metin temsil yönteminin sentiment analizinde daha başarılı olduğunu bulmayı amaçlıyorum.

### 1. Kategorik Verilerin Encode Edilmesi
Çıktı etiketi olan "status" sütunu, kategorik bir değişkendir. Dolayısıyla model eğitiminden önce sayısal hale dönüştürülmesi gerekir. LabelEncoder sınıfı, her bir etiketin (örneğin "Positive", "Negative") sayısal bir değeri karşılık gelmesini sağlar. Bu sayede, makine öğrenmesi modelleri bu etiketleri kullanarak sınıflandırma yapabilir.

### 2. Train-Test Split
Eğitim ve test veri setlerine ayırma işlemi yapılır. train_test_split fonksiyonu ile verilerin %80'i eğitim, %20'si test seti olarak ayrılır.

### 3. Model Eğitimi:
İlk model, TF-IDF vektörleri ile eğitilir. Modelin doğruluğu, test verisi kullanılarak tahmin edilip değerlendirilir. Aşağıdaki metrikler raporlanır:
* F1-Score
* Precision
* Recall

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# 1. Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['status'])

# 2. Bağımsız değişkenler
X_tfidf = np.array(df['TFIDF_Vector'].tolist())
X_word2vec = np.array(df['Word2Vec_Vector'].tolist())

# 3. Veri bölme
X_tfidf_train, X_tfidf_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y)

X_word2vec_train, X_word2vec_test, _, _ = train_test_split(
    X_word2vec, y, test_size=0.2, random_state=42, stratify=y)

# 4. Modelleri tanımlama
tfidf_model = LogisticRegression(max_iter=1000, random_state=42)
word2vec_model = LogisticRegression(max_iter=1000, random_state=42)

# 5. TF-IDF Model Eğitimi
print("TF-IDF Vektörleri ile Model Eğitimi:")
tfidf_model.fit(X_tfidf_train, y_train)
y_pred_tfidf = tfidf_model.predict(X_tfidf_test)
print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf, zero_division=0))

# 6. Word2Vec Model Eğitimi
print("Word2Vec Vektörleri ile Model Eğitimi:")
word2vec_model.fit(X_word2vec_train, y_train)
y_pred_word2vec = word2vec_model.predict(X_word2vec_test)
print("Accuracy (Word2Vec):", accuracy_score(y_test, y_pred_word2vec))
print(classification_report(y_test, y_pred_word2vec, zero_division=0))

TF-IDF Vektörleri ile Model Eğitimi:
Accuracy (TF-IDF): 0.875
              precision    recall  f1-score   support

           0       0.95      0.79      0.86       124
           1       0.93      0.87      0.90       123
           2       0.80      0.95      0.87       153

    accuracy                           0.88       400
   macro avg       0.89      0.87      0.88       400
weighted avg       0.89      0.88      0.88       400

Word2Vec Vektörleri ile Model Eğitimi:
Accuracy (Word2Vec): 0.365
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       124
           1       0.31      0.13      0.18       123
           2       0.37      0.85      0.52       153

    accuracy                           0.36       400
   macro avg       0.23      0.33      0.23       400
weighted avg       0.24      0.36      0.25       400



# SVM Modeli

Logistic Regression'da kullandığım aynı adımları izledim ve SVM Modeli eğittim.

In [ ]:
df_2 = df_copy.copy()

In [ ]:
from sklearn.svm import SVC

# 1. TF-IDF ve Word2Vec vektörlerini numpy array formatına dönüştürme
tfidf_vectors = np.array(df_2['TFIDF_Vector'].tolist())  # TFIDF vektörlerini numpy array'e dönüştürme
word2vec_vectors = np.array(df_2['Word2Vec_Vector'].tolist())  # Word2Vec vektörlerini numpy array'e dönüştürme

# 2. 'status' etiketini alalım ve Label Encoding yapalım
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_2['status'])  # Positive, Neutral gibi etiketleri sayısal hale getirme

# 3. Veriyi eğitim ve test setlerine ayıralım
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(tfidf_vectors, labels, test_size=0.2, random_state=42)
X_train_word2vec, X_test_word2vec, _, _ = train_test_split(word2vec_vectors, labels, test_size=0.2, random_state=42)

# ---------------------------- TF-IDF SVM Modeli ----------------------------
# 4. TF-IDF vektörleri ile SVM modelini oluşturup eğitelim
svm_classifier_tfidf = SVC(kernel='linear')  # Linear kernel kullanıyoruz
svm_classifier_tfidf.fit(X_train_tfidf, y_train)

# 5. TF-IDF modelinin tahminlerini yapalım
y_pred_tfidf = svm_classifier_tfidf.predict(X_test_tfidf)

# 6. TF-IDF modelini değerlendirelim
print("TF-IDF Model Accuracy:", accuracy_score(y_test, y_pred_tfidf))  # Doğruluk oranı
print("TF-IDF Classification Report:\n", classification_report(y_test, y_pred_tfidf, zero_division=0))  # Diğer metrikler

# ---------------------------- Word2Vec SVM Modeli ----------------------------
# 4. Word2Vec vektörleri ile SVM modelini oluşturup eğitelim
svm_classifier_word2vec = SVC(kernel='linear')  # Linear kernel kullanıyoruz
svm_classifier_word2vec.fit(X_train_word2vec, y_train)

# 5. Word2Vec modelinin tahminlerini yapalım
y_pred_word2vec = svm_classifier_word2vec.predict(X_test_word2vec)

# 6. Word2Vec modelini değerlendirelim
print("Word2Vec Model Accuracy:", accuracy_score(y_test, y_pred_word2vec))  # Doğruluk oranı
print("Word2Vec Classification Report:\n", classification_report(y_test, y_pred_word2vec, zero_division=0))  # Diğer metrikler



TF-IDF Model Accuracy: 0.8725
TF-IDF Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.80      0.85       120
           1       0.91      0.88      0.90       122
           2       0.83      0.92      0.87       158

    accuracy                           0.87       400
   macro avg       0.88      0.87      0.87       400
weighted avg       0.88      0.87      0.87       400

Word2Vec Model Accuracy: 0.395
Word2Vec Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       120
           1       0.00      0.00      0.00       122
           2       0.40      1.00      0.57       158

    accuracy                           0.40       400
   macro avg       0.13      0.33      0.19       400
weighted avg       0.16      0.40      0.22       400



# Random Forest Modeli
Önceki adımları takip ederek Random Forest eğittim.

In [ ]:
df_2 = df_copy.copy()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. TF-IDF ve Word2Vec vektörlerini numpy array formatına dönüştürme
tfidf_vectors = np.array(df_2['TFIDF_Vector'].tolist())  # TFIDF vektörlerini numpy array'e dönüştürme
word2vec_vectors = np.array(df_2['Word2Vec_Vector'].tolist())  # Word2Vec vektörlerini numpy array'e dönüştürme

# 2. 'status' etiketini alalım ve Label Encoding yapalım
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_2['status'])  # Positive, Neutral gibi etiketleri sayısal hale getirme

# 3. Veriyi eğitim ve test setlerine ayıralım
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(tfidf_vectors, labels, test_size=0.2, random_state=42)
X_train_w2v, X_test_w2v, y_train, y_test = train_test_split(word2vec_vectors, labels, test_size=0.2, random_state=42)

# 4. TF-IDF ile Random Forest modelini oluşturup eğitelim
rf_tfidf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # 100 ağaçla eğitim yapıyoruz
rf_tfidf_classifier.fit(X_train_tfidf, y_train)

# 5. Word2Vec ile Random Forest modelini oluşturup eğitelim
rf_w2v_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # 100 ağaçla eğitim yapıyoruz
rf_w2v_classifier.fit(X_train_w2v, y_train)

# 6. TF-IDF modelinin tahminlerini yapalım
y_pred_tfidf = rf_tfidf_classifier.predict(X_test_tfidf)

# 7. Word2Vec modelinin tahminlerini yapalım
y_pred_w2v = rf_w2v_classifier.predict(X_test_w2v)

# 8. TF-IDF modelini değerlendirelim
print("TF-IDF Model Accuracy:", accuracy_score(y_test, y_pred_tfidf))  # Doğruluk oranı
print("TF-IDF Classification Report:\n", classification_report(y_test, y_pred_tfidf, zero_division=0))  # Diğer metrikler

# 9. Word2Vec modelini değerlendirelim
print("Word2Vec Model Accuracy:", accuracy_score(y_test, y_pred_w2v))  # Doğruluk oranı
print("Word2Vec Classification Report:\n", classification_report(y_test, y_pred_w2v, zero_division=0))  # Diğer metrikler


TF-IDF Model Accuracy: 0.86
TF-IDF Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.84      0.83       120
           1       0.89      0.87      0.88       122
           2       0.87      0.87      0.87       158

    accuracy                           0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400

Word2Vec Model Accuracy: 0.6575
Word2Vec Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.47      0.52       120
           1       0.81      0.80      0.81       122
           2       0.60      0.68      0.64       158

    accuracy                           0.66       400
   macro avg       0.66      0.65      0.66       400
weighted avg       0.66      0.66      0.65       400



# Naive Bayes Sınıflandırıcı
Önceki adımları takip ederek Naive Bayes eğittim.

In [ ]:
df_2 = df_copy.copy()

In [ ]:
from sklearn.naive_bayes import GaussianNB

# 1. TF-IDF ve Word2Vec vektörlerini numpy array formatına dönüştürme
tfidf_vectors = np.array(df_2['TFIDF_Vector'].tolist())  # TFIDF vektörlerini numpy array'e dönüştürme
word2vec_vectors = np.array(df_2['Word2Vec_Vector'].tolist())  # Word2Vec vektörlerini numpy array'e dönüştürme

# 2. 'status' etiketini alalım ve Label Encoding yapalım
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_2['status'])  # Positive, Neutral gibi etiketleri sayısal hale getirme

# 3. Veriyi eğitim ve test setlerine ayıralım
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(tfidf_vectors, labels, test_size=0.2, random_state=42)
X_train_w2v, X_test_w2v, y_train, y_test = train_test_split(word2vec_vectors, labels, test_size=0.2, random_state=42)

# 4. TF-IDF ile Naive Bayes modelini oluşturup eğitelim
nb_tfidf_classifier = GaussianNB()  # Gaussian Naive Bayes sınıflandırıcı
nb_tfidf_classifier.fit(X_train_tfidf, y_train)

# 5. Word2Vec ile Naive Bayes modelini oluşturup eğitelim
nb_w2v_classifier = GaussianNB()  # Gaussian Naive Bayes sınıflandırıcı
nb_w2v_classifier.fit(X_train_w2v, y_train)

# 6. TF-IDF modelinin tahminlerini yapalım
y_pred_tfidf = nb_tfidf_classifier.predict(X_test_tfidf)

# 7. Word2Vec modelinin tahminlerini yapalım
y_pred_w2v = nb_w2v_classifier.predict(X_test_w2v)

# 8. TF-IDF modelini değerlendirelim
print("TF-IDF Model Accuracy:", accuracy_score(y_test, y_pred_tfidf))  # Doğruluk oranı
print("TF-IDF Classification Report:\n", classification_report(y_test, y_pred_tfidf, zero_division=0))  # Diğer metrikler

# 9. Word2Vec modelini değerlendirelim
print("Word2Vec Model Accuracy:", accuracy_score(y_test, y_pred_w2v))  # Doğruluk oranı
print("Word2Vec Classification Report:\n", classification_report(y_test, y_pred_w2v, zero_division=0))  # Diğer metrikler


TF-IDF Model Accuracy: 0.7575
TF-IDF Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.73      0.75       120
           1       0.65      0.93      0.77       122
           2       0.92      0.65      0.76       158

    accuracy                           0.76       400
   macro avg       0.78      0.77      0.76       400
weighted avg       0.79      0.76      0.76       400

Word2Vec Model Accuracy: 0.35
Word2Vec Classification Report:
               precision    recall  f1-score   support

           0       0.31      0.33      0.32       120
           1       0.36      0.72      0.48       122
           2       0.48      0.08      0.14       158

    accuracy                           0.35       400
   macro avg       0.38      0.38      0.31       400
weighted avg       0.39      0.35      0.30       400



# K-Nearest Neighbors Sınıflandırıcı
Önceki adımları takip ederek K-Nearest Neighbors eğittim.

In [ ]:
df_2 = df_copy.copy()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 1. TF-IDF ve Word2Vec vektörlerini numpy array formatına dönüştürme
tfidf_vectors = np.array(df_2['TFIDF_Vector'].tolist())  # TFIDF vektörlerini numpy array'e dönüştürme
word2vec_vectors = np.array(df_2['Word2Vec_Vector'].tolist())  # Word2Vec vektörlerini numpy array'e dönüştürme

# 2. 'status' etiketini alalım ve Label Encoding yapalım
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_2['status'])  # Positive, Neutral gibi etiketleri sayısal hale getirme

# 3. Veriyi eğitim ve test setlerine ayıralım
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(tfidf_vectors, labels, test_size=0.2, random_state=42)
X_train_w2v, X_test_w2v, y_train, y_test = train_test_split(word2vec_vectors, labels, test_size=0.2, random_state=42)

# 4. TF-IDF ile KNN modelini oluşturup eğitelim
knn_tfidf_classifier = KNeighborsClassifier(n_neighbors=5)  # KNN modelinde k=5 komşu kullanıyoruz
knn_tfidf_classifier.fit(X_train_tfidf, y_train)

# 5. Word2Vec ile KNN modelini oluşturup eğitelim
knn_w2v_classifier = KNeighborsClassifier(n_neighbors=5)  # KNN modelinde k=5 komşu kullanıyoruz
knn_w2v_classifier.fit(X_train_w2v, y_train)

# 6. TF-IDF modelinin tahminlerini yapalım
y_pred_tfidf = knn_tfidf_classifier.predict(X_test_tfidf)

# 7. Word2Vec modelinin tahminlerini yapalım
y_pred_w2v = knn_w2v_classifier.predict(X_test_w2v)

# 8. TF-IDF modelini değerlendirelim
print("TF-IDF Model Accuracy:", accuracy_score(y_test, y_pred_tfidf))  # Doğruluk oranı
print("TF-IDF Classification Report:\n", classification_report(y_test, y_pred_tfidf, zero_division=1))  # Diğer metrikler

# 9. Word2Vec modelini değerlendirelim
print("Word2Vec Model Accuracy:", accuracy_score(y_test, y_pred_w2v))  # Doğruluk oranı
print("Word2Vec Classification Report:\n", classification_report(y_test, y_pred_w2v, zero_division=1))  # Diğer metrikler



TF-IDF Model Accuracy: 0.7125
TF-IDF Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.53      0.66       120
           1       0.56      0.93      0.70       122
           2       0.88      0.68      0.76       158

    accuracy                           0.71       400
   macro avg       0.77      0.71      0.71       400
weighted avg       0.78      0.71      0.71       400

Word2Vec Model Accuracy: 0.4625
Word2Vec Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.38      0.37       120
           1       0.70      0.49      0.58       122
           2       0.42      0.51      0.46       158

    accuracy                           0.46       400
   macro avg       0.49      0.46      0.47       400
weighted avg       0.49      0.46      0.47       400



# LSTM Modeli
Önceki adımları takip ederek Long Short Term Memory modeli eğittim.

In [ ]:
df_2 = df_copy.copy()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. TF-IDF ve Word2Vec vektörlerini numpy array formatına dönüştürme
tfidf_vectors = np.array(df_2['TFIDF_Vector'].tolist())  # TFIDF vektörlerini numpy array'e dönüştürme
word2vec_vectors = np.array(df_2['Word2Vec_Vector'].tolist())  # Word2Vec vektörlerini numpy array'e dönüştürme

# 2. 'status' etiketini alalım ve Label Encoding yapalım
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_2['status'])  # Positive, Neutral gibi etiketleri sayısal hale getirme

# 3. Veriyi eğitim ve test setlerine ayıralım
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(tfidf_vectors, labels, test_size=0.2, random_state=42)
X_train_w2v, X_test_w2v, y_train, y_test = train_test_split(word2vec_vectors, labels, test_size=0.2, random_state=42)

# 4. LSTM modelini hazırlamak için giriş verilerini pad'lemek
# Pad edilmesi gerekebilir çünkü LSTM, sabit boyutta girişler ister
max_sequence_length = max([len(x) for x in tfidf_vectors])  # Verinin en uzun boyutunu alıyoruz
X_train_tfidf = pad_sequences(X_train_tfidf, maxlen=max_sequence_length)
X_test_tfidf = pad_sequences(X_test_tfidf, maxlen=max_sequence_length)

X_train_w2v = pad_sequences(X_train_w2v, maxlen=max_sequence_length)
X_test_w2v = pad_sequences(X_test_w2v, maxlen=max_sequence_length)

# 5. TF-IDF verisi için LSTM modelini oluşturup eğitelim
lstm_tfidf_model = Sequential()
lstm_tfidf_model.add(LSTM(64, activation='relu', input_shape=(max_sequence_length, 1), return_sequences=False))
lstm_tfidf_model.add(Dropout(0.5))
lstm_tfidf_model.add(Dense(32, activation='relu'))
lstm_tfidf_model.add(Dense(len(np.unique(labels)), activation='softmax'))  # sınıf sayısına göre çıkış

lstm_tfidf_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# LSTM için veriyi 3D yapmamız gerekiyor (num_samples, max_sequence_length, 1)
X_train_tfidf_reshaped = np.expand_dims(X_train_tfidf, axis=-1)
X_test_tfidf_reshaped = np.expand_dims(X_test_tfidf, axis=-1)

lstm_tfidf_model.fit(X_train_tfidf_reshaped, y_train, epochs=5, batch_size=64, validation_data=(X_test_tfidf_reshaped, y_test))

# 6. Word2Vec verisi için LSTM modelini oluşturup eğitelim
lstm_w2v_model = Sequential()
lstm_w2v_model.add(LSTM(64, activation='relu', input_shape=(max_sequence_length, 1), return_sequences=False))
lstm_w2v_model.add(Dropout(0.5))
lstm_w2v_model.add(Dense(32, activation='relu'))
lstm_w2v_model.add(Dense(len(np.unique(labels)), activation='softmax'))  # sınıf sayısına göre çıkış

lstm_w2v_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Word2Vec için veriyi 3D yapmamız gerekiyor (num_samples, max_sequence_length, 1)
X_train_w2v_reshaped = np.expand_dims(X_train_w2v, axis=-1)
X_test_w2v_reshaped = np.expand_dims(X_test_w2v, axis=-1)

lstm_w2v_model.fit(X_train_w2v_reshaped, y_train, epochs=5, batch_size=64, validation_data=(X_test_w2v_reshaped, y_test))

# 7. TF-IDF modelinin tahminlerini yapalım
y_pred_tfidf = lstm_tfidf_model.predict(X_test_tfidf_reshaped)
y_pred_tfidf = np.argmax(y_pred_tfidf, axis=1)  # En yüksek olasılığa sahip sınıfı seçiyoruz

# 8. Word2Vec modelinin tahminlerini yapalım
y_pred_w2v = lstm_w2v_model.predict(X_test_w2v_reshaped)
y_pred_w2v = np.argmax(y_pred_w2v, axis=1)  # En yüksek olasılığa sahip sınıfı seçiyoruz

# 9. TF-IDF modelini değerlendirelim
print("TF-IDF Model Accuracy:", accuracy_score(y_test, y_pred_tfidf))  # Doğruluk oranı
print("TF-IDF Classification Report:\n", classification_report(y_test, y_pred_tfidf, zero_division=1))  # Diğer metrikler

# 10. Word2Vec modelini değerlendirelim
print("Word2Vec Model Accuracy:", accuracy_score(y_test, y_pred_w2v))  # Doğruluk oranı
print("Word2Vec Classification Report:\n", classification_report(y_test, y_pred_w2v, zero_division=1))  # Diğer metrikler


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 284ms/step - accuracy: 0.3183 - loss: 1.0985 - val_accuracy: 0.3950 - val_loss: 1.0963
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 152ms/step - accuracy: 0.3690 - loss: 1.0973 - val_accuracy: 0.3950 - val_loss: 1.0940
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 145ms/step - accuracy: 0.3828 - loss: 1.0948 - val_accuracy: 0.3950 - val_loss: 1.0919
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 187ms/step - accuracy: 0.3803 - loss: 1.0939 - val_accuracy: 0.3950 - val_loss: 1.0916
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 171ms/step - accuracy: 0.3619 - loss: 1.0969 - val_accuracy: 0.3950 - val_loss: 1.0915
Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 159ms/step - accuracy: 0.3687 - loss: 1.0983 - val_accuracy: 0.3950 - val_loss: 1.0971
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 203ms/step - accuracy: 0.3788 - loss: 1.0974 - val_accuracy: 0.3950 - val_loss: 1.0958
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 173ms/step - accuracy: 0.3845 - loss: 1.0961 - val_accuracy: 0.3950 - 

# Sonuç

Farklı modellerle yapılan denemelerde, TF-IDF ve Word2Vec tekniklerinin başarı oranları aşağıdaki gibi bulunmuştur.

|                     | TF-IDF | Word2Vec |
|---------------------|--------|----------|
| Logistic Regression | 0.875  | 0.365    |
| SVM                 | 0.8725 | 0.395    |
| Random Forest       | 0.86   | 0.6575   |
| Naive Bayes         | 0.7575 | 0.35     |
| K-Nearest Neighbors | 0.7125 | 0.4625   |
| LSTM                | 0.395  | 0.395    |

Yapılan 6 testin beşinde **TF-IDF** tekniği kullanılarak eğitilen model, Word2Vec ile eğitilen modelden önemli ölçüde daha başarılı olmuştur. Dolayısıyla metinlerde duygu analizi yapmak için TF-IDF'in daha uygun bir vektörleştirme yöntemi olduğu söylenebilir. Test edilen modeller arasında ise **Logistic Regression** ve **SVM** en başarılı sonuçları vermiştir. Ancak modeller eğitilirken hiperparametre optimizasyonu yapılmamıştır. Dolayısıyla, en uygun parametrelerle eğitildiğinde modellerin başarı oranları değişebilir.